<h1>
    The Variational Approximation for Bayesian Inference
</h1>
<p>
    Life after the EM Algorithm
</p>
<br>
<p>
    Dimitris G. Tzikas, Aristidis C. Likas, and Nikolaos P. Galatsanos
</p>
<br>
<br>
<br>
<br>
<br>
<p style="font-size:10px;">CMI Lab Journal Club, 11/22/2019, Andrew Van</p>

<p>This subslide runs a bunch of scripts for this presentation.</p>
<script>window.addEventListener('load', () => { Reveal.configure({ transition: 'fade' }); });</script>

<h1>Motivation</h1>

- Variational Methods?
    - **How does Variational Expectation-Maximization work?**
    - This applies to anything with the word "Variational" (e.g. Variational Autoencoder)
    
- Succinctly...
    - Variational methods allow for the approximation of the posterior of latent variables
    - Derives a lower bound for the marginal likelihood of the observed data

<h1>Introduction</h1>

- Expectation Maximization (EM) is awesome!
    - Applications: Joint problems in image reconstruction, segmentation, registration, etc.
    - We use it to find estimates of model parameters that rely on unobserved latent variables
    - However, limited applicability when encountering complex models
    
- Variational approximation relaxes requirements of EM
    - Can be applied to a wider range of models
    - EM can be viewed as a special case of Variational Bayesian Inference
        - a.k.a Varitational EM (VEM)


<h1>Expectation-Maximization</h1>

We want to find:

$$
\hat{\theta} = \underset{\theta}{\arg\!\max} \; p(x; \theta)
$$

where $x$ are observations, and $\theta$ are parameters.

In EM, we introduce unobserved latent variables (by marginalizing the latent variables):

$$
p(x;\theta) = \int p(x, z; \theta) dz
$$

This expression is known as the **marginal likelihood**.

<h1>Why it's called Bayesian Inference</h1>

Using the Marginal Likelihood and Bayes Rule, we can do inference on posterior of latent variables, $p(z | x; \theta)$

Marginal Likelihood:

$$
p(x; \theta) = \int p(x, z; \theta) dz = \int p(x | z; \theta) p(z; \theta) dz
$$

Bayes Rule:

$$
p(z | x; \theta) = \frac{p(x | z; \theta) p(z; \theta)}{p(x; \theta)}
$$

- However, it's difficult to solve the integral in the Marginal Likelihood term
    - Effort in developing techniques to get around or approximate integral
        - Numerical Sampling (e.g Monte Carlo) or Deterministic Approximation(Variational Methods)

<h1>Bayesian Inference vs. MAP Estimation</h1>

- What is the difference between MAP estimation and Bayesian Inference?
- Both are Bayesian because they place priors on parameters, $\theta$ right?

MAP Estimation:

$$
    \hat{\theta}_{MAP} = \underset{\theta}{\arg\!\max} \; p(x | \theta) p(\theta)
$$

Bayesian Inference:

$$
    p(\theta | x) = \frac{p(x | \theta) p(\theta)}{\int p(x | \theta) p(\theta) d\theta)}
$$

- They are distinctly different in objective!
    - MAP generates a point estimate on $\theta$ (the mode of the posterior) while Bayesian Inference calculates the full posterior distribution.
    - MAP is also known as "Poor Man's Bayesian Inference"

<h1>Evidence Lower Bound (ELBO)</h1>

The marginal likelihood can be rewritten as:

$$
\ln p(x; \theta) = F(q, \theta) + KL(q || p)
$$

with:

$$
F(q, \theta) = \int q(z) \ln \Big( \frac{p(x, z; \theta)}{q(z)} \Big) dz
$$

- Since the KL divergence must be $\geq$ 0, it follows that $\ln p(x; \theta) \geq  F(q, \theta)$
- $F(q, \theta)$ is a lower bound of the log-likelihood (known as the Evidence Lower Bound (ELBO))

<h1>ELBO Derivation</h1>

$$
\ln p(x; \theta) = \int q(z) \ln p(x; \theta) dz \\
= \int q(z) \ln p(x; \theta) dz \\
= \int q(z) \ln \Big( \frac{p(x; \theta) p(z | x; \theta)}{p(z | x; \theta)} \Big) dz \\
= \int q(z) \ln \Big( \frac{p(x, z; \theta)}{p(z | x; \theta)} \Big) dz \\
= \int q(z) \ln \Big( \frac{p(x, z; \theta) q(z)}{p(z | x; \theta) q(z)} \Big) dz \\
= \int q(z) \ln \Big( \frac{p(x, z; \theta)}{q(z)} \Big) dz - \int q(z) \ln \Big( \frac{p(z | x; \theta)}{q(z)} \Big) dz \\
= F(q, \theta) + KL(q || p)
$$

<h1>Maximizing the ELBO</h1>

We want to maximize the ELBO:

$$
F(q, \theta) = \int q(z) \ln \Big( \frac{p(x, z; \theta)}{q(z)} \Big) dz
$$

- We use a two-step process to maximize the lower bound (given starting parameters $\theta^{OLD}$)
    - Step 1: Maximize $F(q, \theta^{OLD})$ w/ respect to $q(z)$
    - Step 2: Maximize $F(q, \theta)$ w/ respect to $\theta$ to generate $\theta^{NEW}$

- The lower bound is the same as maximizing the log likelihood when $KL(q || p) = 0$
    - This implies that $q(z) = p(z | x; \theta^{OLD})$

<h1>Maximizing the ELBO (continued)</h1>

Setting the latent posterior ($p(z | x; \theta^{OLD})$) to be $q(z)$, we derive the familiar form for the EM algorithm:

$$
F(q, \theta) = \int p(z | x; \theta^{OLD}) \ln p(x, z; \theta) dz - \int p(z | x; \theta^{OLD}) \ln p(z | x; \theta^{OLD}) dz \\
= \langle \ln p(x, z; \theta)\rangle_{p(z | x; \theta^{OLD})} + constant \\
= Q(\theta, \theta^{OLD}) + constant
$$

- The EM algorithm
    - E-step: Compute $Q(\theta, \theta^{OLD})$
    - M-step: Find $\theta$ that maximizes $Q(\theta, \theta^{OLD})$

- The trick is that we must explicilty know $p(z | x; \theta)$ to compute $Q(\theta, \theta^{OLD})$ (or $F(q, \theta)$)
    - Not always applicable in all problems, and we can't use EM :(

<h1>Variational Approximation</h1>

- We can bypass knowing $p(z | x; \theta)$ exactly by assuming $q(z)$ has a specific form and optimize over $F(q, \theta)$ using calculus of variations
    - Thus the name "Variational Approximation"

- Common form used: Mean Field approximation:

$$
q(z) = \prod_{i=1}^{M} q_{i}(z_{i})
$$

<h1>Variational Approximation (Continued)</h1>

Applying the form $q(z) = \prod_{i=1}^{M} q_{i}(z_{i})$ specified by the mean field approximation, we get the optimal $q(z)$ that maximizes the lower bound:

$$
q_{j}^{*}(z_j) = \frac{\exp(\langle \ln p(x, z; \theta)\rangle_{i \neq j})}{\int \exp(\langle \ln p(x, z; \theta)\rangle_{i \neq j}) dz_{j}} 
$$

for each latent variable $j = 1, ..., M$

<h1>Mean Field Solution Derivation</h1>

$$
F(q, \theta) = \int q(z) \ln \Big( \frac{p(x, z; \theta)}{q(z)} \Big) dz \\
= \int \prod_i q(z_i) \ln p(x, z; \theta) dz - \sum_i \int q(z_i) \ln q(z_i) dz_i \\
= \int q(z_j) \int \Big( \prod_{i \neq j} q(z_i) \ln p(x, z; \theta) \Big) \prod_{i \neq j} dz_i dz_j - \int q(z_j) \ln q(z_j) dz_j - \sum_{i \neq j} \int q(z_i) \ln q(z_i) dz_i \\
= \int q(z_j) \ln \Big( \frac{\exp(\langle \ln p(x, z; \theta)\rangle_{i \neq j})}{q(z_j)} \Big) dz_j - \sum_{i \neq j} \int q(z_i) \ln q(z_i) dz_i
$$

<h1>Mean Field Solution Derivation (Continued)</h1>

$$
= \int q(z_j) \ln \Big( \frac{\tilde{p}(x,z_{j}; \theta)}{q(z_j)} \Big) dz_j - \sum_{i \neq j} \int q(z_i) \ln q(z_i) dz_i \\
= - KL(q_j || \tilde{p}) - \sum_{i \neq j} \int q(z_i) \ln q(z_i) dz_i
$$

where:

$$
\ln \tilde{p}(x,z_{j}; \theta) = \langle \ln p(x, z; \theta)\rangle_{i \neq j} = \int \ln p(x, z; \theta) \prod_{i \neq j} (q_{i}dz_{i}
$$

<h1>Mean Field Solution Derivation (Continued 2)</h1>

Like before, $F(q, \theta)$ is maximized when the KL divergence is 0, which occurs when $q_{j}(z_{j}) = \tilde{p}(x,z_{j}; \theta)$, so:

$$
\ln q_{j}^{*}(z_{j}) = \langle \ln p(x, z; \theta)\rangle_{i \neq j} + constant
$$

The additive constant can be obtained through normalization, so the final solution form is:

$$
q_{j}^{*}(z_j) = \frac{\exp(\langle \ln p(x, z; \theta)\rangle_{i \neq j})}{\int \exp(\langle \ln p(x, z; \theta)\rangle_{i \neq j}) dz_{j}} 
$$

<h1>Variational EM</h1>

- With the mean field form of $q(z)$, the Variation EM (VEM) algorithm can be summarized as:
    - E-step: Evaluate $q^{NEW}(z)$ to maximize $F(q,\theta^{OLD})$
    - M-step: Find $\theta^{NEW}$ that maximizes $F(q^{NEW},\theta)$
    
- A common case is where the model only contains latent variables and no parameters
    - In this case, the VEM algorithm only has a Expectation step and no Maximization step

<h1>A quick background on graphical models</h1>

<div style="display:flex;flex-direction:row;margin-top:10px;">
    <div>
        <img style="width:auto;" src="images/graph_models.png">
    </div>
    <div style="margin-left:10px;">
        <ul>
            <li>Graphical models represent dependencies among random variables and parameters</li>
            <ul>
                <li>Double Circle: Observations</li>
                <li>Circle: Latent Variables</li>
                <li>Squares: Parameters</li>
            </ul>
            <li>Example (left):</li>
            <ul>
                <li>Each graph node $(s)$ has a set of parents $\pi(s)$ they are conditioned on: $p(x_{s} | x_{\pi(s)})$</li>
                <li>Joint distribution can be defined by: $p(x; \theta) = \prod_{s} p(x_{s} | x_{\pi(s)})$</li>
                <li>So the joint distribution of the left graph can be defined as:</li>
                $$p(a,b,c,d; \theta) = p(a; \theta_{1}) p(b | a; \theta_{2}) p(c | a; \theta_{3}) p(d | b,c; \theta_{4})$$
            </ul>
            <li>In VEM, do the E-step on circles (Latent Variables) and M-step on squares (parameters)</li>
        </ul>
    </div>
</div>

<h1>Example 1 (Linear Regression)</h1>

- [GitHub](https://github.com/vanandrew/variational_em_demo/blob/master/regression-example.ipynb)

<div style="text-align:center;display:flex;justify-content:center;">
    <img style="height:35vh;width:auto;" src="images/linear_regression.png">
</div>

<h1>Example 2 (Gaussian Mixture Model)</h1>

- [GitHub](https://github.com/vanandrew/variational_em_demo/blob/master/gmm-example.ipynb)

<div style="display:flex;flex-direction:row;justify-content:center;">
    <div style="padding:10px;text-align:center">
        <div style="display:flex;justify-content:center;">
            <img style="height:15vh;width:auto;" src="images/GMM.png">
        </div>
        <p>GMM Model using EM</p>
    </div>
    <div style="padding:10px;text-align:center;">
        <div style="display:flex;justify-content:center;">
            <img style="height:15vh;width:auto;" src="images/VGMM.png">
        </div>
        <p>Full Bayesian GMM Model using VEM</p>
    </div>
</div>

<h1>Useful Links and References</h1>

This paper: https://ieeexplore.ieee.org/document/4644060

Derivations for linear regression solutions: https://arxiv.org/abs/1301.3838

Latex derivations: https://chrischoy.github.io/research/Expectation-Maximization-and-Variational-Inference/
